In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from IPython.display import display

current_dir = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(current_dir, "../../../../../"))

sys.path.append(os.path.join(PROJECT_ROOT, "shared_utilities"))
from metrics import *



In [2]:
ptid_list = [540, 544, 552, 559, 563, 567, 570, 575, 584, 588, 591, 596]

## **Batch  Size = 64**

## **Over and Undersampled Data**

In [4]:
dir_64 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/learning_rate/000001")

dict_64 = {}

detailed_results_df_64 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_64, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_64[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_64, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_64 = pd.concat([detailed_results_df_64, detailed_df], axis=0)
    detailed_results_df_64.reset_index(drop=True, inplace=True)
detailed_results_df_64.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.539494,-0.407716,0.000000,0.000000,115.0,124.260130,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.610648,-0.403161,-0.014231,0.000911,110.0,124.580185,-1.0,0.064010,A,B,AP,eu,540
2,0,2,-0.639109,-0.394774,-0.005692,0.001677,108.0,125.169530,-0.4,0.117870,A,A,AP,eu,540
3,0,3,-0.653340,-0.403660,-0.002846,-0.001777,107.0,124.545120,-0.2,-0.124882,A,A,AP,eu,540
4,0,4,-0.667570,-0.397537,-0.002846,0.001225,106.0,124.975390,-0.2,0.086054,A,A,AP,eu,540


In [5]:
all_64_dfs = list(dict_64.values())

In [6]:
all_64_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_64_dfs]
aggregate_64_df = sum(all_64_value_only_dfs)
# add glycaemic region column
aggregate_64_df["Glycaemic Region"] = all_64_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_64_df = aggregate_64_df[["Glycaemic Region"] + list(aggregate_64_df.columns[:-1])]

aggregate_64_df["AP:EP"] = aggregate_64_df["AP"] / aggregate_64_df["EP"]

aggregate_64_df["AP%"] = aggregate_64_df["AP"] / aggregate_64_df["Count"] * 100
aggregate_64_df["BE%"] = aggregate_64_df["BE"] / aggregate_64_df["Count"] * 100
aggregate_64_df["EP%"] = aggregate_64_df["EP"] / aggregate_64_df["Count"] * 100

display(aggregate_64_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11173,379,1569,13121,7.121096,85.153571,2.888499,11.957930
1,eu,307104,16670,3864,327638,79.478261,93.732717,5.087932,1.179350
2,hyper,172705,9181,6555,188441,26.347063,91.649376,4.872082,3.478542
3,overall,490982,26230,11988,529200,40.956123,92.778156,4.956538,2.265306


In [7]:
for ptid, df in dict_64.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_64[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1916,50,247,2213,7.757085,86.579304,2.259376,11.161319
1,eu,29397,2128,251,31776,117.119522,92.513218,6.696878,0.789904
2,hyper,12948,971,884,14803,14.647059,87.468756,6.559481,5.971762
3,overall,44261,3149,1382,48792,32.026773,90.713642,6.453927,2.832432


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,127,0,33,160,3.848485,79.375000,0.000000,20.625000
1,eu,31041,1044,144,32229,215.562500,96.313879,3.239319,0.446803
2,hyper,15237,606,296,16139,51.476351,94.411054,3.754879,1.834067
3,overall,46405,1650,473,48528,98.107822,95.625206,3.400099,0.974695


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,88,7,1,96,88.000000,91.666667,7.291667,1.041667
1,eu,29570,1912,612,32094,48.316993,92.135602,5.957500,1.906898
2,hyper,25810,1477,619,27906,41.696284,92.489070,5.292769,2.218161
3,overall,55468,3396,1232,60096,45.022727,92.298988,5.650958,2.050053


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1587,74,460,2121,3.450000,74.823197,3.488920,21.687883
1,eu,35311,2700,521,38532,67.775432,91.640714,7.007163,1.352123
2,hyper,14147,876,700,15723,20.210000,89.976468,5.571456,4.452077
3,overall,51045,3650,1681,56376,30.365854,90.543848,6.474386,2.981765


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1061,5,254,1320,4.177165,80.378788,0.378788,19.242424
1,eu,33964,1506,119,35589,285.411765,95.433982,4.231645,0.334373
2,hyper,12344,820,615,13779,20.071545,89.585601,5.951085,4.463314
3,overall,47369,2331,988,50688,47.944332,93.452099,4.598722,1.949179


## **Batch Size = 32**

In [8]:
dir_32 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/batch_size/batch_size_32")

dict_32 = {}

detailed_results_df_32 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_32, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_32[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_32, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_32 = pd.concat([detailed_results_df_32, detailed_df], axis=0)
    detailed_results_df_32.reset_index(drop=True, inplace=True)
detailed_results_df_32.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,0.172042,0.000072,0.000000,0.000000,165.0,152.91556,0.0,0.000000,A,A,AP,eu,540
1,0,1,0.271657,-0.029286,0.019923,-0.005872,172.0,150.85257,1.4,-0.412598,A,B,AP,eu,540
2,0,2,0.371272,-0.061877,0.019923,-0.006518,179.0,148.56240,1.4,-0.458033,A,B,AP,eu,540
3,0,3,0.456657,-0.096958,0.017077,-0.007016,185.0,146.09723,1.2,-0.493033,A,B,AP,hyper,540
4,0,4,0.470887,-0.118444,0.002846,-0.004297,186.0,144.58737,0.2,-0.301974,B,A,AP,hyper,540


In [9]:
all_32_dfs = list(dict_32.values())

In [10]:
all_32_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_32_dfs]
aggregate_32_df = sum(all_32_value_only_dfs)
# add glycaemic region column
aggregate_32_df["Glycaemic Region"] = all_32_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_32_df = aggregate_32_df[["Glycaemic Region"] + list(aggregate_32_df.columns[:-1])]

aggregate_32_df["AP:EP"] = aggregate_32_df["AP"] / aggregate_32_df["EP"]

aggregate_32_df["AP%"] = aggregate_32_df["AP"] / aggregate_32_df["Count"] * 100
aggregate_32_df["BE%"] = aggregate_32_df["BE"] / aggregate_32_df["Count"] * 100
aggregate_32_df["EP%"] = aggregate_32_df["EP"] / aggregate_32_df["Count"] * 100

display(aggregate_32_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11406,387,1328,13121,8.588855,86.929350,2.949470,10.121180
1,eu,306822,16471,4345,327638,70.614960,93.646647,5.027195,1.326159
2,hyper,172562,9363,6516,188441,26.482812,91.573490,4.968664,3.457846
3,overall,490790,26221,12189,529200,40.264993,92.741875,4.954837,2.303288


In [11]:
for ptid, df in dict_32.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_32[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1978,83,152,2213,13.013158,89.380931,3.750565,6.868504
1,eu,29242,2076,458,31776,63.847162,92.025428,6.533233,1.441339
2,hyper,12967,994,842,14803,15.400238,87.597109,6.714855,5.688036
3,overall,44187,3153,1452,48792,30.431818,90.561977,6.462125,2.975898


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,138,6,16,160,8.625000,86.250000,3.750000,10.000000
1,eu,30928,1056,245,32229,126.236735,95.963263,3.276552,0.760185
2,hyper,15416,518,205,16139,75.200000,95.520169,3.209616,1.270215
3,overall,46482,1580,466,48528,99.746781,95.783877,3.255852,0.960270


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,80,6,10,96,8.000000,83.333333,6.250000,10.416667
1,eu,29750,1817,527,32094,56.451613,92.696454,5.661494,1.642051
2,hyper,25600,1554,752,27906,34.042553,91.736544,5.568695,2.694761
3,overall,55430,3377,1289,60096,43.002327,92.235756,5.619342,2.144901


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1765,119,237,2121,7.447257,83.215464,5.610561,11.173975
1,eu,35172,2563,797,38532,44.130489,91.279975,6.651614,2.068411
2,hyper,14045,1003,675,15723,20.807407,89.327736,6.379190,4.293074
3,overall,50982,3685,1709,56376,29.831480,90.432099,6.536469,3.031432


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1052,7,261,1320,4.030651,79.696970,0.530303,19.772727
1,eu,33964,1510,115,35589,295.339130,95.433982,4.242884,0.323134
2,hyper,12355,809,615,13779,20.089431,89.665433,5.871253,4.463314
3,overall,47371,2326,991,50688,47.801211,93.456045,4.588857,1.955098


## **Batch Size = 128**

In [12]:
dir_128 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/batch_size/batch_size_128")

dict_128 = {}

detailed_results_df_128 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_128, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_128[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_128, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_128 = pd.concat([detailed_results_df_128, detailed_df], axis=0)
    detailed_results_df_128.reset_index(drop=True, inplace=True)
detailed_results_df_128.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.027188,-0.481582,0.000000,0.000000,151.0,119.069496,0.0,0.000000,B,A,AP,eu,540
1,0,1,-0.141034,-0.512920,-0.022769,-0.006268,143.0,116.867380,-1.6,-0.440424,A,B,AP,eu,540
2,0,2,-0.226418,-0.708253,-0.017077,-0.039067,137.0,103.141230,-1.2,-2.745230,A,B,AP,eu,540
3,0,3,-0.397187,-0.810408,-0.034154,-0.020431,125.0,95.962740,-2.4,-1.435698,A,A,AP,eu,540
4,0,4,-0.454110,-0.940356,-0.011385,-0.025990,121.0,86.831200,-0.8,-1.826308,B,B,AP,eu,540


In [13]:
all_128_dfs = list(dict_128.values())

In [14]:
all_128_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_128_dfs]
aggregate_128_df = sum(all_128_value_only_dfs)
# add glycaemic region column
aggregate_128_df["Glycaemic Region"] = all_128_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_128_df = aggregate_128_df[["Glycaemic Region"] + list(aggregate_128_df.columns[:-1])]

aggregate_128_df["AP:EP"] = aggregate_128_df["AP"] / aggregate_128_df["EP"]

aggregate_128_df["AP%"] = aggregate_128_df["AP"] / aggregate_128_df["Count"] * 100
aggregate_128_df["BE%"] = aggregate_128_df["BE"] / aggregate_128_df["Count"] * 100
aggregate_128_df["EP%"] = aggregate_128_df["EP"] / aggregate_128_df["Count"] * 100

display(aggregate_128_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11586,364,1171,13121,9.894108,88.301197,2.774179,8.924625
1,eu,307739,16391,3508,327638,87.724914,93.926529,5.002777,1.070694
2,hyper,171794,9568,7079,188441,24.268117,91.165935,5.077451,3.756613
3,overall,491119,26323,11758,529200,41.768923,92.804044,4.974112,2.221844


In [15]:
for ptid, df in dict_128.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_128[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1963,34,216,2213,9.087963,88.703118,1.536376,9.760506
1,eu,29489,2118,169,31776,174.491124,92.802744,6.665408,0.531848
2,hyper,12759,1012,1032,14803,12.363372,86.191988,6.836452,6.971560
3,overall,44211,3164,1417,48792,31.200423,90.611166,6.484670,2.904165


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,130,0,30,160,4.333333,81.250000,0.000000,18.750000
1,eu,30978,1139,112,32229,276.589286,96.118403,3.534084,0.347513
2,hyper,15187,617,335,16139,45.334328,94.101245,3.823037,2.075717
3,overall,46295,1756,477,48528,97.054507,95.398533,3.618530,0.982938


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,95,0,1,96,95.000000,98.958333,0.000000,1.041667
1,eu,29897,1825,372,32094,80.368280,93.154484,5.686421,1.159095
2,hyper,25565,1553,788,27906,32.442893,91.611123,5.565111,2.823765
3,overall,55557,3378,1161,60096,47.852713,92.447085,5.621006,1.931909


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1770,136,215,2121,8.232558,83.451202,6.412070,10.136728
1,eu,35164,2548,820,38532,42.882927,91.259213,6.612686,2.128101
2,hyper,14120,969,634,15723,22.271293,89.804745,6.162946,4.032309
3,overall,51054,3653,1669,56376,30.589575,90.559813,6.479708,2.960480


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1175,32,113,1320,10.398230,89.015152,2.424242,8.560606
1,eu,33614,1566,409,35589,82.185819,94.450532,4.400236,1.149232
2,hyper,12603,731,445,13779,28.321348,91.465273,5.305175,3.229552
3,overall,47392,2329,967,50688,49.009307,93.497475,4.594776,1.907749


In [16]:
display(aggregate_32_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11406,387,1328,13121,8.588855,86.929350,2.949470,10.121180
1,eu,306822,16471,4345,327638,70.614960,93.646647,5.027195,1.326159
2,hyper,172562,9363,6516,188441,26.482812,91.573490,4.968664,3.457846
3,overall,490790,26221,12189,529200,40.264993,92.741875,4.954837,2.303288


## **Batch Size = 256**

In [17]:
dir_256 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/batch_size/batch_size_256")

dict_256 = {}

detailed_results_df_256 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_256, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_256[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_256, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_256 = pd.concat([detailed_results_df_256, detailed_df], axis=0)
    detailed_results_df_256.reset_index(drop=True, inplace=True)
detailed_results_df_256.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.012957,0.141815,0.000000,0.000000,152.0,162.87592,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.012957,0.198857,0.000000,0.011408,152.0,166.88428,0.0,0.801671,A,A,AP,eu,540
2,0,2,-0.069880,0.244570,-0.011385,0.009143,148.0,170.09656,-0.8,0.642456,A,B,AP,eu,540
3,0,3,-0.169495,0.251943,-0.019923,0.001475,141.0,170.61467,-1.4,0.103623,A,B,AP,eu,540
4,0,4,-0.269110,0.307721,-0.019923,0.011156,134.0,174.53420,-1.4,0.783904,B,lD,BE,eu,540


In [18]:
all_256_dfs = list(dict_256.values())

In [19]:
all_256_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_256_dfs]
aggregate_256_df = sum(all_256_value_only_dfs)
# add glycaemic region column
aggregate_256_df["Glycaemic Region"] = all_256_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_256_df = aggregate_256_df[["Glycaemic Region"] + list(aggregate_256_df.columns[:-1])]

aggregate_256_df["AP:EP"] = aggregate_256_df["AP"] / aggregate_256_df["EP"]

aggregate_256_df["AP%"] = aggregate_256_df["AP"] / aggregate_256_df["Count"] * 100
aggregate_256_df["BE%"] = aggregate_256_df["BE"] / aggregate_256_df["Count"] * 100
aggregate_256_df["EP%"] = aggregate_256_df["EP"] / aggregate_256_df["Count"] * 100

display(aggregate_256_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11769,398,954,13121,12.336478,89.695907,3.033305,7.270787
1,eu,307219,16433,3986,327638,77.074511,93.767817,5.015596,1.216587
2,hyper,172366,9448,6627,188441,26.009657,91.469479,5.013771,3.516751
3,overall,491354,26279,11567,529200,42.478949,92.848450,4.965797,2.185752


In [20]:
for ptid, df in dict_256.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_256[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1941,32,240,2213,8.087500,87.708992,1.446001,10.845007
1,eu,29492,2154,130,31776,226.861538,92.812185,6.778701,0.409114
2,hyper,12698,1056,1049,14803,12.104862,85.779909,7.133689,7.086401
3,overall,44131,3242,1419,48792,31.100070,90.447204,6.644532,2.908264


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,129,2,29,160,4.448276,80.625000,1.250000,18.125000
1,eu,31127,1061,41,32229,759.195122,96.580719,3.292066,0.127215
2,hyper,15103,610,426,16139,35.453052,93.580767,3.779664,2.639569
3,overall,46359,1673,496,48528,93.465726,95.530415,3.447494,1.022090


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,82,3,11,96,7.454545,85.416667,3.125000,11.458333
1,eu,30096,1788,210,32094,143.314286,93.774537,5.571135,0.654328
2,hyper,25522,1526,858,27906,29.745921,91.457034,5.468358,3.074608
3,overall,55700,3317,1079,60096,51.621872,92.685037,5.519502,1.795461


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1866,144,111,2121,16.810811,87.977369,6.789250,5.233380
1,eu,35108,2436,988,38532,35.534413,91.113879,6.322018,2.564103
2,hyper,14105,1042,576,15723,24.487847,89.709343,6.627234,3.663423
3,overall,51079,3622,1675,56376,30.494925,90.604158,6.424720,2.971122


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1172,52,96,1320,12.208333,88.787879,3.939394,7.272727
1,eu,33396,1650,543,35589,61.502762,93.837984,4.636264,1.525752
2,hyper,12794,642,343,13779,37.300292,92.851441,4.659264,2.489295
3,overall,47362,2344,982,50688,48.230143,93.438289,4.624369,1.937342


## **Batch Size Comparison**

In [29]:
display(aggregate_32_df)

display(aggregate_64_df)

display(aggregate_128_df)

display(aggregate_256_df)


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11406,387,1328,13121,8.588855,86.929350,2.949470,10.121180
1,eu,306822,16471,4345,327638,70.614960,93.646647,5.027195,1.326159
2,hyper,172562,9363,6516,188441,26.482812,91.573490,4.968664,3.457846
3,overall,490790,26221,12189,529200,40.264993,92.741875,4.954837,2.303288


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11173,379,1569,13121,7.121096,85.153571,2.888499,11.957930
1,eu,307104,16670,3864,327638,79.478261,93.732717,5.087932,1.179350
2,hyper,172705,9181,6555,188441,26.347063,91.649376,4.872082,3.478542
3,overall,490982,26230,11988,529200,40.956123,92.778156,4.956538,2.265306


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11586,364,1171,13121,9.894108,88.301197,2.774179,8.924625
1,eu,307739,16391,3508,327638,87.724914,93.926529,5.002777,1.070694
2,hyper,171794,9568,7079,188441,24.268117,91.165935,5.077451,3.756613
3,overall,491119,26323,11758,529200,41.768923,92.804044,4.974112,2.221844


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11769,398,954,13121,12.336478,89.695907,3.033305,7.270787
1,eu,307219,16433,3986,327638,77.074511,93.767817,5.015596,1.216587
2,hyper,172366,9448,6627,188441,26.009657,91.469479,5.013771,3.516751
3,overall,491354,26279,11567,529200,42.478949,92.848450,4.965797,2.185752


In [30]:
results_list = [aggregate_32_df, aggregate_64_df, aggregate_128_df, aggregate_256_df]
results_dict = {}

for i, table in enumerate(results_list):
    #delete last row
    table = table[:-1]
    average_ap_ep = table["AP:EP"].mean()
    results_dict[f"Table {i+1}"] = average_ap_ep

for key, value in results_dict.items():
    print(f"{key}: {value}")

Table 1: 35.22887556210994
Table 2: 37.64880680655278
Table 3: 40.629046349066854
Table 4: 38.47354874569228


In [ ]:
best_hypo_32 = []
best_hypo_64 = []
best_hypo_128 = []
best_hypo_256 = []


for ptid in ptid_list:

    hypo_dict = {}
    hypo_32 = dict_32[ptid].loc[dict_32[ptid]["Glycaemic Region"] == "hypo"]
    hypo_32_apep = hypo_32["AP:EP"].values[0]
    hypo_dict["32"] = hypo_32_apep

    hypo_64 = dict_64[ptid].loc[dict_64[ptid]["Glycaemic Region"] == "hypo"]
    hypo_64_apep = hypo_64["AP:EP"].values[0]
    hypo_dict["64"] = hypo_64_apep

    hypo_128 = dict_128[ptid].loc[dict_128[ptid]["Glycaemic Region"] == "hypo"]
    hypo_128_apep = hypo_128["AP:EP"].values[0]
    hypo_dict["128"] = hypo_128_apep

    hypo_256 = dict_256[ptid].loc[dict_256[ptid]["Glycaemic Region"] == "hypo"]
    hypo_256_apep = hypo_256["AP:EP"].values[0]
    hypo_dict["256"] = hypo_256_apep



    best_hypo = max(hypo_dict, key=hypo_dict.get)
    if best_hypo == "32":
        best_hypo_32.append(ptid)
    elif best_hypo == "64":
        best_hypo_64.append(ptid)
    elif best_hypo == "128":
        best_hypo_128.append(ptid)
    elif best_hypo == "256":
        best_hypo_256.append(ptid)
    else:
        print("Error: No best hypo found for patient ID:", ptid)

print("Batch size 32 results in best Hypo AP:EP for ptid: ", best_hypo_32)
print("Batch size 64 results in best Hypo AP:EP for ptid: ", best_hypo_64)
print("Batch size 128 results in best Hypo AP:EP for ptid: ", best_hypo_128)
print("Batch size 256 results in best Hypo AP:EP for ptid: ", best_hypo_256)




Batch size 32 results in best Hypo AP:EP for ptid:  [540, 544, 563, 575]
Batch size 64 results in best Hypo AP:EP for ptid:  [567, 584]
Batch size 128 results in best Hypo AP:EP for ptid:  [588]
Batch size 256 results in best Hypo AP:EP for ptid:  [552, 559, 570, 591, 596]
Batch size 512 results in best Hypo AP:EP for ptid:  [563, 567, 575, 596]


In [ ]:
best_overall_32 = []
best_overall_64 = []
best_overall_128 = []
best_overall_256 = []


for ptid in ptid_list:

    overall_dict = {}
    overall_32 = dict_32[ptid].loc[dict_32[ptid]["Glycaemic Region"] == "overall"]
    overall_32_apep = overall_32["AP:EP"].values[0]
    overall_dict["32"] = overall_32_apep

    overall_64 = dict_64[ptid].loc[dict_64[ptid]["Glycaemic Region"] == "overall"]
    overall_64_apep = overall_64["AP:EP"].values[0]
    overall_dict["64"] = overall_64_apep

    overall_128 = dict_128[ptid].loc[dict_128[ptid]["Glycaemic Region"] == "overall"]
    overall_128_apep = overall_128["AP:EP"].values[0]
    overall_dict["128"] = overall_128_apep

    overall_256 = dict_256[ptid].loc[dict_256[ptid]["Glycaemic Region"] == "overall"]
    overall_256_apep = overall_256["AP:EP"].values[0]
    overall_dict["256"] = overall_256_apep



    best_overall = max(overall_dict, key=overall_dict.get)
    if best_overall == "32":
        best_overall_32.append(ptid)
    elif best_overall == "64":
        best_overall_64.append(ptid)
    elif best_overall == "128":
        best_overall_128.append(ptid)
    elif best_overall == "256":
        best_overall_256.append(ptid)
    else:
        print("Error: No best hypo found for patient ID:", ptid)
print("Batch size 32 results in best Overall AP:EP for ptid: ", best_overall_32)
print("Batch size 64 results in best Overall AP:EP for ptid: ", best_overall_64)
print("Batch size 128 results in best Overall AP:EP for ptid: ", best_overall_128)
print("Batch size 256 results in best Overall AP:EP for ptid: ", best_overall_256)

Batch size 32 results in best Overall AP:EP for ptid:  [544]
Batch size 64 results in best Overall AP:EP for ptid:  [540]
Batch size 128 results in best Overall AP:EP for ptid:  [596]
Batch size 256 results in best Overall AP:EP for ptid:  [575]
Batch size 512 results in best Overall AP:EP for ptid:  [552, 559, 563, 567, 570, 584, 588, 591]
